# Assignment 3 - Ensemble Methods #

Welcome to your third assignment. This exercise will test your understanding on Ensemble Methods.

In [6]:
# Always run this cell
import numpy as np
import pandas as pd

# USE THE FOLLOWING RANDOM STATE FOR YOUR CODE
RANDOM_STATE = 42

## Download the Dataset ##
Download the dataset using the following cell or from this [link](https://github.com/sakrifor/public/tree/master/machine_learning_course/EnsembleDataset) and put the files in the same folder as the .ipynb file. 
In this assignment you are going to work with a dataset originated from the [ImageCLEFmed: The Medical Task 2016](https://www.imageclef.org/2016/medical) and the **Compound figure detection** subtask. The goal of this subtask is to identify whether a figure is a compound figure (one image consists of more than one figure) or not. The train dataset consits of 4197 examples/figures and each figure has 4096 features which were extracted using a deep neural network. The *CLASS* column represents the class of each example where 1 is a compoung figure and 0 is not. 


In [ ]:
import urllib.request
url_train = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/train_set.csv'
filename_train = 'train_set.csv'
urllib.request.urlretrieve(url_train, filename_train)
url_test = 'https://github.com/sakrifor/public/raw/master/machine_learning_course/EnsembleDataset/test_set_noclass.csv'
filename_test = 'test_set_noclass.csv'
urllib.request.urlretrieve(url_test, filename_test)

In [7]:
# Run this cell to load the data
train_set = pd.read_csv("train_set.csv").sample(frac=1).reset_index(drop=True)
train_set.head()
X = train_set.drop(columns=['CLASS'])
y = train_set['CLASS'].values

## 1.0 Testing different ensemble methods ##
In this part of the assignment you are asked to create and test different ensemble methods using the train_set.csv dataset. You should use **10-fold cross validation** for your tests and report the average f-measure and accuracy of your models.

### !!! Use n_jobs=-1 where is posibble to use all the cores of a machine for running your tests ###

### 1.1 Voting ###
Create a voting classifier which uses three estimators/classifiers. Test both soft and hard voting and choose the best one.

In [8]:
# BEGIN CODE HERE
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_validate


# Preprocess the data by MinMax scaling
transformer = MinMaxScaler().fit(X)
X = pd.DataFrame(transformer.transform(X))
y_train = pd.DataFrame(y)

# Pick best parameters for classifier number 1
# param_grid = [{'criterion': ["gini", "entropy"],
#                'max_depth': list(range(1, 38)),
#                'min_samples_split': [2, 3, 4, 5]
#                }]
#
# grid_search = GridSearchCV(DecisionTreeClassifier(random_state = RANDOM_STATE), param_grid, cv=10, scoring='f1', return_train_score=True, verbose=3)
# grid_search.fit(X, y)
# print(grid_search.best_params_)
# CV Result: {'criterion': 'entropy', 'max_depth': 4, 'min_samples_split': 2}

# Using the CV best parameters for classifier number 1
cls1 = DecisionTreeClassifier(random_state=RANDOM_STATE, criterion='entropy', max_depth=4, min_samples_split=2) # Classifier #1

# Pick best parameters for classifier number 2
# param_grid = [{'criterion': ["gini", "entropy"]}]
#
# grid_search = GridSearchCV(DecisionTreeClassifier(random_state = RANDOM_STATE), param_grid, cv=10, scoring='f1', return_train_score=True, verbose=3)
# grid_search.fit(X, y)
# print(grid_search.best_params_)
# CV Result: {'criterion': 'entropy'}

# Using the CV best parameters for classifier number 2
cls2 = DecisionTreeClassifier(random_state=RANDOM_STATE, criterion='entropy') # Classifier #2

# Pick best parameters for classifier number 3
# param_grid = [{'n_neighbors': [3, 5],
#                'weights': ["uniform", "distance"],
#                'p': [1, 2] # L1 or L2
#                }]
#
# grid_search = GridSearchCV(KNeighborsClassifier(n_jobs=-1), param_grid, cv=10, scoring='f1', return_train_score=True, verbose=3)
# grid_search.fit(X, y)
# print(grid_search.best_params_)
# CV Result: {'n_neighbors': 5, 'p': 2, 'weights': 'distance'}

# Using the CV best parameters for classifier number 3
cls3 = KNeighborsClassifier(n_neighbors=5, weights='distance', p=2, n_jobs=-1) # Classifier #3

clrs = [('dt4', cls1), ('dt', cls2), ('knn', cls3)]

# Pick between hard and soft voting
# param_grid = [{'voting': ["hard", "soft"]}]
#
# grid_search = GridSearchCV(VotingClassifier(estimators=clrs, n_jobs=-1), param_grid, cv=10, scoring='f1', return_train_score=True, verbose=3)
# grid_search.fit(X, y)
# print(grid_search.best_params_)
# CV Result: {'voting': 'hard'}

# # # # # This code would probably (can't be sure) produce better results, but it would take an eternity to execute # # # # #
# cls1 = DecisionTreeClassifier(random_state=RANDOM_STATE, criterion='entropy', max_depth=4, min_samples_split=2) # Classifier #1
# cls2 = DecisionTreeClassifier(random_state=RANDOM_STATE, criterion='entropy') # Classifier #2
# cls3 = KNeighborsClassifier(n_neighbors=5, weights='distance', p=2, n_jobs=-1) # Classifier #3
# clrs = [('dt', cls1), ('dta', cls2), ('knn', cls3)]
# vcls = VotingClassifier(estimators=clrs, n_jobs=-1) # Voting Classifier
#
# param_grid = [{'dt__criterion': ["gini", "entropy"],
#                'dt__max_depth': list(range(1, 38)),
#                'dt__min_samples_split': [2, 3, 4, 5],
#                'dta__criterion': ["gini", "entropy"],
#                'knn__n_neighbors': [3, 5],
#                'knn__weights': ["uniform", "distance"],
#                'knn__p': [1, 2] # L1 or L2
#                }]
#
# grid_search = GridSearchCV(estimator=vcls, param_grid=param_grid, cv=10, scoring='f1', return_train_score=True, verbose=3, n_jobs=-1)
# grid_search.fit(X, y)
# print(grid_search.best_params_)

vcls = VotingClassifier(estimators=clrs, voting='hard', n_jobs=-1)

scores = cross_validate(estimator=vcls, X=X, y=y, cv=10,
                        scoring=['f1', 'accuracy'],
                        return_train_score=True,
                        verbose=3,
                        n_jobs=-1)

avg_fmeasure = np.average(scores['test_f1']) # The average f-measure
avg_accuracy = np.average(scores['test_accuracy']) # The average accuracy

#END CODE HERE

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   22.7s remaining:   52.9s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   24.9s remaining:   10.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   32.7s finished


In [9]:
print("Classifier:")
print(vcls)
print("F1-Score:{} & Accuracy:{}".format(avg_fmeasure,avg_accuracy))

Classifier:
VotingClassifier(estimators=[('dt4',
                              DecisionTreeClassifier(criterion='entropy',
                                                     max_depth=4,
                                                     random_state=42)),
                             ('dt',
                              DecisionTreeClassifier(criterion='entropy',
                                                     random_state=42)),
                             ('knn',
                              KNeighborsClassifier(n_jobs=-1,
                                                   weights='distance'))],
                 n_jobs=-1)
F1-Score:0.8115998157012017 & Accuracy:0.7638157745198317


### 1.2 Stacking ###
Create a stacking classifier which uses two estimators/classifiers. Try different classifiers for the combination of the initial classifiers. Report your results in the following cell.

In [10]:
# BEGIN CODE HERE
from sklearn.ensemble import StackingClassifier

# cls1 = DecisionTreeClassifier(random_state=RANDOM_STATE, criterion='entropy', max_depth=4, min_samples_split=2) # Classifier #1
# cls2 = DecisionTreeClassifier(random_state=RANDOM_STATE, criterion='entropy') # Classifier #2
# clrs = [('cls1', cls1), ('cls2', cls2)]
# scls = StackingClassifier(estimators=clrs, n_jobs=-1, verbose=3) # Stacking Classifier
#
# scores = cross_validate(estimator=scls, X=X, y=y, cv=10,
#                         scoring=['f1', 'accuracy'],
#                         return_train_score=True,
#                         verbose=3,
#                         n_jobs=-1)
#
# avg_fmeasure = np.average(scores['test_f1']) # The average f-measure
# avg_accuracy = np.average(scores['test_accuracy']) # The average accuracy

cls1 = DecisionTreeClassifier(random_state=RANDOM_STATE, criterion='entropy', max_depth=4, min_samples_split=2) # Classifier #1
cls2 = KNeighborsClassifier(n_neighbors=5, weights='distance', p=2, n_jobs=-1) # Classifier #3
clrs = [('cls1', cls1), ('cls2', cls2)]
scls = StackingClassifier(estimators=clrs, n_jobs=-1, verbose=3) # Stacking Classifier

scores = cross_validate(estimator=scls, X=X, y=y, cv=10,
                        scoring=['f1', 'accuracy'],
                        return_train_score=True,
                        verbose=3,
                        n_jobs=-1)

avg_fmeasure = np.average(scores['test_f1']) # The average f-measure
avg_accuracy = np.average(scores['test_accuracy'])

# cls1 = DecisionTreeClassifier(random_state=RANDOM_STATE, criterion='entropy') # Classifier #2
# cls2 = KNeighborsClassifier(n_neighbors=5, weights='distance', p=2, n_jobs=-1) # Classifier #3
# clrs = [('cls1', cls1), ('cls2', cls2)]
# scls = StackingClassifier(estimators=clrs, n_jobs=-1, verbose=3) # Stacking Classifier
#
# scores = cross_validate(estimator=scls, X=X, y=y, cv=10,
#                         scoring=['f1', 'accuracy'],
#                         return_train_score=True,
#                         verbose=3,
#                         n_jobs=-1)
#
# avg_fmeasure = np.average(scores['test_f1']) # The average f-measure
# avg_accuracy = np.average(scores['test_accuracy'])

#END CODE HERE

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   40.6s remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   41.3s remaining:   17.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   54.6s finished


In [11]:
print("Classifier:")
print(scls)
print("F1-Score:{} & Accuracy:{}".format(avg_fmeasure,avg_accuracy))

Classifier:
StackingClassifier(estimators=[('cls1',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=4,
                                                       random_state=42)),
                               ('cls2',
                                KNeighborsClassifier(n_jobs=-1,
                                                     weights='distance'))],
                   n_jobs=-1, verbose=3)
F1-Score:0.8309012150241951 & Accuracy:0.7950448914649393


### 1.3 Report the results ###
Report the results of your experiments in the following cell. How did you choose your initial classifiers? 

Firstly, a basic MinMaxScaler is used to preprocess the data.

For the selection of classifiers the following strategy is used, in an attempt to produce reasonably good results.

SVMs seem unreasonable to use in ensembles due to the long training time, and the high scores they offer.
Probably best to use them by themselves.

Gaussian Classifiers seem to not offer good enough scores in comparison to other classification algorithms.

Use the DecisionTreeClassifier that produces the best score.
Use the DecisionTreeClassifier that is fully adapted to the training data.
Use the KNeighborsClassifier in order to capture the spatial information that characterizes the data without the
overfitting nature that follows the fully adapted DecisionTreeClassifier.

After the above process is done, the hard vs soft voting dilemma is solved.
Finally, the voting classifier is trained, and the predictions are made.
The above method results in a f1-score approximating 0.81 and accuracy roughly equal to 0.76.

For the stacking classifier, all possible combinations of the three classifiers that were used previously are tried and
the one that grants us with the best results was kept. It seems a quite reasonable strategy.

The above process resulted in a f1-score approximating 0.83 and accuracy roughly equal to 0.79 when using KNeighborsClassifier
combined with the best-scoring DecisionTreeClassifier.

## 2.0 Randomization ##

**2.1** You are asked to create three ensembles of decision trees where each one uses a different method for producing homogeneous ensembles. Compare them with a simple decision tree classifier and report your results in the dictionaries (dict) below using as key the given name of your classifier and as value the f1/accuracy score. The dictionaries should contain four different elements.  

In [12]:
# BEGIN CODE HERE
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier

# ens1 = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=RANDOM_STATE), n_estimators=100, random_state=RANDOM_STATE, n_jobs=4, verbose=3)
# ens1_scores = cross_validate(estimator=ens1, X=X, y=y, cv=10,
#                         scoring=['f1', 'accuracy'],
#                         return_train_score=True,
#                         verbose=3,
#                         n_jobs=4)
#
# ens2 = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=4, verbose=3)
# ens2_scores = cross_validate(estimator=ens2, X=X, y=y, cv=10,
#                         scoring=['f1', 'accuracy'],
#                         return_train_score=True,
#                         verbose=3,
#                         n_jobs=4)
#
# ens3 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=RANDOM_STATE), n_estimators=100, random_state=RANDOM_STATE)
# ens3_scores = cross_validate(estimator=ens3, X=X, y=y, cv=10,
#                         scoring=['f1', 'accuracy'],
#                         return_train_score=True,
#                         verbose=3,
#                         n_jobs=4)
#
# tree = DecisionTreeClassifier(random_state=RANDOM_STATE) # Classifier #1
# tree_scores = cross_validate(estimator=tree, X=X, y=y, cv=10,
#                         scoring=['f1', 'accuracy'],
#                         return_train_score=True,
#                         verbose=3,
#                         n_jobs=4)
#
# f_measures = {'Ensemble with Bagging': np.average(ens1_scores['test_f1']),
#               'Ensemble with Random Forest': np.average(ens2_scores['test_f1']),
#               'Ensemble with AdaBoost': np.average(ens3_scores['test_f1']),
#               'Simple Decision': np.average(tree_scores['test_f1'])}
# accuracies = {'Ensemble with Bagging': np.average(ens1_scores['test_accuracy']),
#               'Ensemble with Random Forest': np.average(ens2_scores['test_accuracy']),
#               'Ensemble with AdaBoost': np.average(ens3_scores['test_accuracy']),
#               'Simple Decision': np.average(tree_scores['test_accuracy'])}
# Example f_measures = {'Simple Decision':0.8551, 'Ensemble with random ...': 0.92, ...}

bcls = DecisionTreeClassifier(random_state=RANDOM_STATE, criterion='entropy', max_depth=4, min_samples_split=2) # Classifier #1

ens1 = BaggingClassifier(base_estimator=bcls, n_estimators=100, random_state=RANDOM_STATE, n_jobs=4, verbose=3)
ens1_scores = cross_validate(estimator=ens1, X=X, y=y, cv=10,
                        scoring=['f1', 'accuracy'],
                        return_train_score=True,
                        verbose=3,
                        n_jobs=4)

ens2 = RandomForestClassifier(criterion='entropy', max_depth=4, min_samples_split=2, n_estimators=100, random_state=RANDOM_STATE, n_jobs=4, verbose=3)
ens2_scores = cross_validate(estimator=ens2, X=X, y=y, cv=10,
                        scoring=['f1', 'accuracy'],
                        return_train_score=True,
                        verbose=3,
                        n_jobs=4)

ens3 = AdaBoostClassifier(base_estimator=bcls, n_estimators=100, random_state=RANDOM_STATE)
ens3_scores = cross_validate(estimator=ens3, X=X, y=y, cv=10,
                        scoring=['f1', 'accuracy'],
                        return_train_score=True,
                        verbose=3,
                        n_jobs=4)

tree = bcls
tree_scores = cross_validate(estimator=tree, X=X, y=y, cv=10,
                        scoring=['f1', 'accuracy'],
                        return_train_score=True,
                        verbose=3,
                        n_jobs=4)

f_measures = {'Ensemble with Bagging': np.average(ens1_scores['test_f1']),
              'Ensemble with Random Forest': np.average(ens2_scores['test_f1']),
              'Ensemble with AdaBoost': np.average(ens3_scores['test_f1']),
              'Simple Decision': np.average(tree_scores['test_f1'])}
accuracies = {'Ensemble with Bagging': np.average(ens1_scores['test_accuracy']),
              'Ensemble with Random Forest': np.average(ens2_scores['test_accuracy']),
              'Ensemble with AdaBoost': np.average(ens3_scores['test_accuracy']),
              'Simple Decision': np.average(tree_scores['test_accuracy'])}
#END CODE HERE

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:  4.4min remaining:  1.9min
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  5.5min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:    3.8s remaining:    1.6s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    5.1s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:  7.8min remaining:  3.3min
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed: 10.7min finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:    4.7s remaining:    2.0s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    6.7s finished


In [13]:
print(ens1)
print(ens2)
print(ens3)
print(tree)
for name,score in f_measures.items():
    print("Classifier:{} -  F1:{}".format(name,score))
for name,score in accuracies.items():
    print("Classifier:{} -  Accuracy:{}".format(name,score))

BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                        max_depth=4,
                                                        random_state=42),
                  n_estimators=100, n_jobs=4, random_state=42, verbose=3)
RandomForestClassifier(criterion='entropy', max_depth=4, n_jobs=4,
                       random_state=42, verbose=3)
AdaBoostClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                         max_depth=4,
                                                         random_state=42),
                   n_estimators=100, random_state=42)
DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=42)
Classifier:Ensemble with Bagging -  F1:0.80528595843536
Classifier:Ensemble with Random Forest -  F1:0.800471607847261
Classifier:Ensemble with AdaBoost -  F1:0.8034443552382895
Classifier:Simple Decision -  F1:0.7760530031602773
Classif

**2.2** Describe your classifiers and your results.

At first, the three different methods for producing homogeneous ensembles (Bagging, Random Forest, Boosting), using 100
estimators, are compared with each other and also with a simple Decision Tree Classifier, without fine-tuning the
algorithm parameters. The results are presented in the table below. Bagging and Random Forest methodologies far
outperform the Boosting algorithm and the Simple Tree.

| Algorithm     |      F1 Score      |   Accuracy Score   |
|---------------|:------------------:|:------------------:|
| Bagging       | 0.8479842034261897 | 0.8141072849187407 |
| Random Forest | 0.8475386243179933 | 0.8102932151380839 |
| Boosting      | 0.7370851894386801 | 0.6928037276963291 |
| Simple Tree   | 0.7393819455067371 | 0.6944635754062961 |

In the second experiment, the above process is followed, but the Decision Tree used is the fine-tuned one. The results
show that when the tree used is the one that produces the best score, the difference between the different methods for
producing homogeneous ensembles are minimal, but still outperforming the simple tree. Boosting is greatly benefited by
using a fine-tuned model since it staring point is far better than the one when using the tree without the right
parameters. Bagging and Random Forest are not performing quite well when the fine-tuned model is used due to the fact
that they have no ability to exploit partial adaptation to the data since the best scoring tree has limited depth.

| Algorithm     |      F1 Score      |   Accuracy Score   |
|---------------|:------------------:|:------------------:|
| Bagging       | 0.8088751928182969 | 0.7540373906125696 |
| Random Forest |  0.80231569382201  |  0.728073644732356 |
| Boosting      |  0.806041713353633 | 0.7683509489714739 |
| Simple Tree   | 0.7609822052165569 | 0.7056665530173882 |


**2.3** Increasing the number of estimators in a bagging classifier can drastically increase the training time of a classifier. Is there any solution to this problem? Can the same solution be applied to boosting classifiers?

The main solution for the problem is called parallelization. Using the n_jobs parameter of sklearn's method allows for
n number of jobs to run in parallel for both fit and predict. That way both training and predicting are tremendously
sped up. However, that parameter cannot be used for boosting classifiers since there is a linear connection between
each and every estimator running. The first classifiers need to be trained in order for the second one to start its own
training process. Due to that fact, its apparent that parallelization cannot be used for this type of ensembles.

## 3.0 Creating the best classifier ##

**3.1** In this part of the assignment you are asked to train the best possible ensemble! Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code. Can you achieve an accuracy over 83-84%?

In [14]:
# BEGIN CODE HERE
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier

best_cls = GradientBoostingClassifier(random_state=RANDOM_STATE, verbose=4, n_estimators=100, learning_rate=0.5, max_depth=9, min_samples_split=3)

# param_grid = [{'learning_rate': [0.1, 0.3, 0.5, 0.7, 0.9],
#                'max_depth': list(range(1, 10)),
#                'min_samples_split': list(range(1, 5))
#                }]
#
# grid_search = GridSearchCV(estimator=best_cls, param_grid=param_grid, cv=3, scoring='accuracy', return_train_score=True, verbose=3, n_jobs=6)
# grid_search.fit(X, y)
# print(grid_search.best_params_)
# CV Result: {'max_depth': 4, 'min_samples_split': 3, 'learning_rate': 0.3}

scores = cross_validate(estimator=best_cls, X=X, y=y, cv=10,
                        scoring=['f1', 'accuracy'],
                        return_train_score=True,
                        n_jobs=4,
                        verbose=2)

best_fmeasure = np.average(scores['test_f1']) # The average f-measure
best_accuracy = np.average(scores['test_accuracy'])

# END CODE HERE

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed: 16.2min finished


In [15]:
print("Classifier:")
print(best_cls)
print("F1-Score:{} & Accuracy:{}".format(best_fmeasure,best_accuracy))

Classifier:
GradientBoostingClassifier(learning_rate=0.5, max_depth=9, min_samples_split=3,
                           random_state=42, verbose=4)
F1-Score:0.8649371747864608 & Accuracy:0.8364996022275258


**3.2** Describe the process you followed to achieve this result. How did you choose your classifier and your parameters and why. Report the f-measure & accuracy (10-fold cross validation) of your final classifier and results of classifiers you tried in the cell following the code.

After trying different ensemble combinations, limited to using 100 estimators, the one that showed the most promising
results was the one using Gradient Boosting after fine-tuning its parameters using 10-Fold cross validation. Increasing
the number of estimator to 250 can result in an improvement of the models accuracy and f1-score, but a constant number
of 100 estimators was kept in order to make the ensemble comparison an easier process.

| Gradient Boosting Parameters |      F1 Score      |   Accuracy Score   |
|---------------|:------------------:|:------------------:|
| learning_rate=0.5, max_depth=9, min_samples_split=3, n_estimators=100 | 0.8649371747864608 | 0.8364996022275258 |

**3.3** Create a classifier that is going to be used in production - in a live system. Use the *test_set_noclass.csv* to make predictions. Store the predictions in a list.  

In [16]:
# BEGIN CODE HERE
cls = best_cls.fit(X,y)
X_test = pd.DataFrame(transformer.transform(pd.read_csv("test_set_noclass.csv")))
predictions = best_cls.predict(X_test)

#END CODE HERE

      Iter       Train Loss   Remaining Time 
         1           0.8308            4.17m
         2           0.5678            4.18m
         3           0.4032            4.19m
         4           0.2938            4.23m
         5           0.2215            4.50m
         6           0.1801            4.61m
         7           0.1482            4.65m
         8           0.1200            4.58m
         9           0.1028            4.48m
        10           0.0877            4.39m
        11           0.0771            4.30m
        12           0.0674            4.23m
        13           0.0594            4.17m
        14           0.0536            4.10m
        15           0.0473            4.03m
        16           0.0416            3.97m
        17           0.0379            3.91m
        18           0.0345            3.85m
        19           0.0322            3.79m
        20           0.0287            3.75m
        21           0.0260            3.73m
        2

In [17]:
print(cls)
print(predictions)

GradientBoostingClassifier(learning_rate=0.5, max_depth=9, min_samples_split=3,
                           random_state=42, verbose=4)
[1 0 1 ... 1 1 1]


LEAVE HERE ANY COMMENTS ABOUT YOUR CLASSIFIER

#### This following cell will not be executed. The test_set.csv with the classes will be made available after the deadline and this cell is for testing purposes!!! Do not modify it! ###

In [ ]:
from sklearn.metrics import f1_score,accuracy_score
final_test_set = pd.read_csv('test_set.csv')
ground_truth = final_test_set['CLASS']
print("Accuracy:{}".format(accuracy_score(predictions,ground_truth)))
print("F1-Score:{}".format(f1_score(predictions,ground_truth)))